# import the necessary package 

In [1]:
import random 
import numpy as np

## Input and massage transformation 

Input the checking massage, in this program although people to check the transportation of (1)4-digit binary code bits

In [2]:
#define the code bits that we want to check    
code_bit=input('Please input a 4 digit binary code:')
#define the vector generator function
def vector_gen(code_bit):
    #check whether the input is a 4-digit code
    if len(code_bit) != 4:
        raise ValueError('Invaild input:please input a 4-digit code.')
    #check whether code bits are binary 
    elif (code_bit.count('0')+code_bit.count('1')) != 4:
        raise ValueError('Invaild input: please input a binary code.')
    #change the format of input into vector
    vector_c=np.array([int(i) for i in code_bit])
    print('The input code bits are:', vector_c)
    return  vector_c

Please input a 4 digit binary code:1001


In [3]:
vector_c=vector_gen(code_bit)

The input code bits are: [1 0 0 1]


# Encode
Define the generator massage which can use to generate the hamming code

In [4]:
#define the encode function
def encode(vector_c):
    #use the numpy array to define the Genetrator matrix
    g_matrix=np.array ([[1,1,1,0,0,0,0],[1,0,0,1,1,0,0],[0,1,0,1,0,1,0],[1,1,0,1,0,0,1]])
    #print('the genetor matrix is',g_matrix)
    #the vector mutiplt the genetrator matrix will be the 7-digit hamming code 
    hamming=vector_c.dot(g_matrix)
    #print ('the vector_matrix product is', hamming)
    #define the overall parity bit 
    ov_par=0
    for i in hamming:
        ov_par=ov_par+i
    hamming=np.insert(hamming,0,ov_par)
    # transform the result into mod 2 expression
    hamming_c=np.array([i%2 for i in hamming])
    print('the hamming code is', hamming_c[1:])
    print('the parity bits are', hamming_c[[1,2,4]])
    print('the overall parity bit is', hamming_c[0])
    #return the hamming code we get 
    return hamming_c

In [5]:
hamming_c=encode(vector_c)

the hamming code is [0 0 1 1 0 0 1]
the parity bits are [0 0 1]
the overall parity bit is 1


# Error Generator 
To test whether the parity check can find the error hear we define a error generator function 
In this program we have 2 genetor (1) the bit_flip function which can choose the bit you want to change, and (2) ran_bit_flip function which can randomly change a bit in the hammimg code
ps. only need to choose one when do the test, or it might generator more than 1 error, affecting the function of the hamming code program

(1)changing bit function

In [6]:
#define the function which can change the nth bit in hamming code
def bit_flip(hamming_c,*bits):
    #check whether the input is bigger than 7 
    for bit in bits:
        if int(bit) > 7 or int(bit)<1:
            raise ValueError ('invaild input: please enter an integer between 1 and 7')
             
        #change the bit from 1 to 0 or from 0 to 1
        else:
            hamming_c[bit]=1-hamming_c[bit]
        
    print('the position of flipped bits:', bits, ';', 'Error code:', hamming_c[1:])
    #return the error code
    return hamming_c

In [7]:
#here suppose to change 2 bits but only change one 
hamming_c=bit_flip(hamming_c,6,7)

the position of flipped bits: (6, 7) ; Error code: [0 0 1 1 0 1 0]


(2) random error generator

In [8]:
#define the ran_bit_flip function
def transportation(hamming_c):
    #generate an random error in hamming code
    flipped_bit=random.randint(1,14)
    #for the number bigger than 7, we assume the transportatiom is successful
    if flipped_bit > 7:
        print('Transportation in progress......\n','......\n','Transportation Successful!No error occured.')
        return hamming_c
    #for the number between 1 and 7, the function will generate the error at that position
    else:
        print('Transportation in progress......\n','......\n','Transportation Failded. The flipped bit is at position:', flipped_bit )
        #flip the bit
        hamming_c[flipped_bit]=1-hamming_c[flipped_bit]
        print('The error code is :', hamming_c[1:])
        #reurn the error codes
        return hamming_c

In [47]:
hamming_c=transportation(hamming_c)

Transportation in progress......
 ......
 Transportation Failded. The flipped bit is at position: 7
The error code is : [1 0 1 0 1 0 0]


# Parity check 


In [9]:
 def par_check(hamming_c):   
    #define the checking matrix 
    h_matrix=([[1,0,0,0],[1,1,0,0],[1,0,1,0],[1,1,1,0],[1,0,0,1],[1,1,0,1],[1,0,1,1],[1,1,1,1]])
    #print ('the parity check matrix is', h_matrix)
    #use the hamming code to multiply the checking matrix to do the parity check  
    vector_re=hamming_c.dot(h_matrix)
    #tranform the format into the binary expression 
    parity_check=np.array([i%2 for i in vector_re])
    print('the result of parity check is', parity_check[1:])        
    return parity_check

In [10]:
parity_check=par_check(hamming_c)

the result of parity check is [1 0 0]


# Correction 
Still thinking how to determine that is not error, one error (with location), and potential 2 errors .
And do the corretion on one error 

In [11]:
#define the correction code 
def correction(hamming_c, parity_check):
    #identify the error position 
    error_position=parity_check[1]+parity_check[2]*2+parity_check[3]*4
    #detect the whether the error occurred
    if error_position !=0 and parity_check[0]==0:
            print('There might be more than an error.')
            return 'There might be more than an errors.'
    elif error_position !=0:
        #correct the error
        hamming_c[error_position]=1-hamming_c[error_position]
        print('The error is at position', error_position)
        print ('The original code is:', hamming_c[1:])
    else: 
        print ('No error occurred during the transpotation.')
    return hamming_c

In [12]:
hamming_c=correction(hamming_c, parity_check)
hamming_c

There might be more than an error.


'There might be more than an errors.'

# Decode

In [13]:
def decode(hamming_c):
    if type(hamming_c) != str:
        # define the decode matrix 
        r_matrix=np.array([[0,0,0,0],[0,0,0,0],[0,0,0,0],[1,0,0,0],[0,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]])
        #print('the decode matrix is', r_matrix)
        #use the encode hamming matrix mutiply with the decode matrix to get the original matrix
        hamming_re=hamming_c.dot(r_matrix)
        print(hamming_re)
        code_bit_re=np.array([i%2 for i in hamming_re])
        print('the original code bits are', code_bit_re)
        return code_bit_re
    else:
        print('There are more than an error occurred during the transportation.\n Unable to decode.')
        return 'There are more than an error occurred during the transportation.\n Unable to decode.'

In [14]:
code_bit_re=decode(hamming_c)

There are more than an error occurred during the transportation.
 Unable to decode.


# Test session 1
Here we will test the hamming code encoder and decoder without the error ocurred during the transportaion


In [15]:
#test_1 - invaild input (3 digit input)
vector_c=vector_gen('111')
print(vector_c)


ValueError: Invaild input:please input a 4-digit code.

In [16]:
#test_2- invaild input(non binary input)
vector_c=vector_gen('1333')
print(vector_c)

ValueError: Invaild input: please input a binary code.

In [18]:
#test_3-vaild input (0000)
test_3=vector_gen('0000')
test_3=encode(test_3)
par_result=par_check(test_3)
test_3=correction(test_3, par_result)
test_3=decode(test_3)

The input code bits are: [0 0 0 0]
the hamming code is [0 0 0 0 0 0 0]
the parity bits are [0 0 0]
the overall parity bit is 0
the result of parity check is [0 0 0]
No error occurred during the transpotation.
[0 0 0 0]
the original code bits are [0 0 0 0]


In [19]:
#test_4- vaild input (1111)
test_4=vector_gen('1111')
test_4=encode(test_4)
par_result=par_check(test_4)
test_4=correction(test_4, par_result)
test4=decode(test_4)

The input code bits are: [1 1 1 1]
the hamming code is [1 1 1 1 1 1 1]
the parity bits are [1 1 1]
the overall parity bit is 1
the result of parity check is [0 0 0]
No error occurred during the transpotation.
[1 1 1 1]
the original code bits are [1 1 1 1]


In [20]:
#test_5- vaild input (1101)
test_5=vector_gen('1101')
test_5=encode(test_5)
par_result=par_check(test_5)
test_5=correction(test_5, par_result)
test_5=decode(test_5)

The input code bits are: [1 1 0 1]
the hamming code is [1 0 1 0 1 0 1]
the parity bits are [1 0 0]
the overall parity bit is 0
the result of parity check is [0 0 0]
No error occurred during the transpotation.
[1 1 0 1]
the original code bits are [1 1 0 1]


# Test Session 2
This session will test the hamming code encoder, decoder, and correction function with the error occurred 


In [21]:
#test_1- vaild input (1111) with 1 error occurred at position 3
test1=vector_gen('1111')
test1=encode(test1)
test1=bit_flip(test1,3)
par_result=par_check(test1)
test1=correction(test1, par_result)
test1=decode(test1)

The input code bits are: [1 1 1 1]
the hamming code is [1 1 1 1 1 1 1]
the parity bits are [1 1 1]
the overall parity bit is 1
the position of flipped bits: (3,) ; Error code: [1 1 0 1 1 1 1]
the result of parity check is [1 1 0]
The error is at position 3
The original code is: [1 1 1 1 1 1 1]
[1 1 1 1]
the original code bits are [1 1 1 1]


In [22]:
#test_2- vaild input (0000) with 1 error occurred at position 1
test2=vector_gen('0000')
test2=encode(test2)
test2=bit_flip(test2,1)
par_result=par_check(test2)
test2=correction(test2, par_result)
test2=decode(test2)

The input code bits are: [0 0 0 0]
the hamming code is [0 0 0 0 0 0 0]
the parity bits are [0 0 0]
the overall parity bit is 0
the position of flipped bits: (1,) ; Error code: [1 0 0 0 0 0 0]
the result of parity check is [1 0 0]
The error is at position 1
The original code is: [0 0 0 0 0 0 0]
[0 0 0 0]
the original code bits are [0 0 0 0]


In [23]:
#test_3- vaild input (1101) with 2 error occurred in position3,4 
test3=vector_gen('1101')
test3=encode(test3)
test3=bit_flip(test3,3,4)
par_result=par_check(test3)
test3=correction(test3, par_result)
test3=decode(test3)

The input code bits are: [1 1 0 1]
the hamming code is [1 0 1 0 1 0 1]
the parity bits are [1 0 0]
the overall parity bit is 0
the position of flipped bits: (3, 4) ; Error code: [1 0 0 1 1 0 1]
the result of parity check is [1 1 1]
There might be more than an error.
There are more than an error occurred during the transportation.
 Unable to decode.


In [24]:
#test_4- vaild input (1001) with 3 error occurred in position3,4,5 
test4=vector_gen('1001')
test4=encode(test4)
test4=bit_flip(test4,3,4,5)
par_result=par_check(test4)
test4=correction(test4, par_result)
test4=decode(test4)

The input code bits are: [1 0 0 1]
the hamming code is [0 0 1 1 0 0 1]
the parity bits are [0 0 1]
the overall parity bit is 1
the position of flipped bits: (3, 4, 5) ; Error code: [0 0 0 0 1 0 1]
the result of parity check is [0 1 0]
The error is at position 2
The original code is: [0 1 0 0 1 0 1]
[0 1 0 1]
the original code bits are [0 1 0 1]


In [25]:
#test_5- vaild input (1001) with 4 error occurred in position3,5,6,7
test5=vector_gen('1101')
test5=encode(test5)
test5=bit_flip(test5,3,5,6,7)
par_result=par_check(test5)
test5=correction(test5, par_result)
test5=decode(test5)

The input code bits are: [1 1 0 1]
the hamming code is [1 0 1 0 1 0 1]
the parity bits are [1 0 0]
the overall parity bit is 0
the position of flipped bits: (3, 5, 6, 7) ; Error code: [1 0 0 0 0 1 0]
the result of parity check is [1 1 1]
There might be more than an error.
There are more than an error occurred during the transportation.
 Unable to decode.


For the two error situtation, hamming code cannot locate the actual fliped code.Since the parity check only allow to place one fliped bit, the two error will influence each other in the parity check.
Therefore, in the end, we will get another position to change, although it can still make the parity check back to 0, but it will distort the original massage.
By adding the overall parity bit we can detect the even number errors, but for the 3 error transportation, it cannot detect.


# Simulation Program 
In this program, user can enter the 4-digit binary code. Then the program will simulate the code transportation scenario.
In the setting, the transportation sucessful rate is 50%, which means it has 50% possibility to generate the error.
Then, the program will do the error detection and automated correction. Finally, the program will decode the hamming code and recover it to the original massage.

In [26]:
def hamming_program(code_bit):
    vector_c=vector_gen(code_bit)
    hamming_c=encode(vector_c)
    hamming_c=transportation(hamming_c)    
    parity_check=par_check(hamming_c)
    hamming_c=correction(hamming_c, parity_check)
    code_bit_re=decode(hamming_c)
    return hamming_c

In [27]:
#test_1 - vaild input (4 digit input:1111)
code_bit=input('Please input a 4 digit binary code:')
test1=hamming_program(code_bit)

Please input a 4 digit binary code:1111
The input code bits are: [1 1 1 1]
the hamming code is [1 1 1 1 1 1 1]
the parity bits are [1 1 1]
the overall parity bit is 1
Transportation in progress......
 ......
 Transportation Failded. The flipped bit is at position: 1
The error code is : [0 1 1 1 1 1 1]
the result of parity check is [1 0 0]
The error is at position 1
The original code is: [1 1 1 1 1 1 1]
[1 1 1 1]
the original code bits are [1 1 1 1]


In [28]:
#test_2 - vaild input (4 digit input:0000)
code_bit=input('Please input a 4 digit binary code:')
test2=hamming_program(code_bit)

Please input a 4 digit binary code:0000
The input code bits are: [0 0 0 0]
the hamming code is [0 0 0 0 0 0 0]
the parity bits are [0 0 0]
the overall parity bit is 0
Transportation in progress......
 ......
 Transportation Failded. The flipped bit is at position: 6
The error code is : [0 0 0 0 0 1 0]
the result of parity check is [0 1 1]
The error is at position 6
The original code is: [0 0 0 0 0 0 0]
[0 0 0 0]
the original code bits are [0 0 0 0]


In [29]:
#test_3 - vaild input (4 digit input:1101)
code_bit=input('Please input a 4 digit binary code:')
test3=hamming_program(code_bit)

Please input a 4 digit binary code:1101
The input code bits are: [1 1 0 1]
the hamming code is [1 0 1 0 1 0 1]
the parity bits are [1 0 0]
the overall parity bit is 0
Transportation in progress......
 ......
 Transportation Successful!No error occured.
the result of parity check is [0 0 0]
No error occurred during the transpotation.
[1 1 0 1]
the original code bits are [1 1 0 1]


In [30]:
#test 4-vaild input (4 digit input)
code_bit=input('Please input a 4 digit binary code:')
test4=hamming_program(code_bit)

Please input a 4 digit binary code:hkjh


ValueError: Invaild input: please input a binary code.